<a href="https://colab.research.google.com/github/Ser9ibus/myStrArt/blob/main/myStrArt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np

# Invert grayscale image
def invertImage(image):
    return (255-image)

# Apply circular mask to image
def maskImage(image, radius):
    y, x = np.ogrid[-radius:radius + 1, -radius:radius + 1]
    mask = x**2 + y**2 > radius**2
    image[mask] = 0

    return image
    
def pinCoords(radius, nPins=200, offset=0, x0=None, y0=None):
    alpha = np.linspace(0 + offset, 2*np.pi + offset, nPins + 1)

    if (x0 == None) or (y0 == None):
        x0 = radius + 1
        y0 = radius + 1

    coords = []
    for angle in alpha[0:-1]:
        x = int(x0 + radius*np.cos(angle))
        y = int(y0 + radius*np.sin(angle))

        coords.append((y, x))
    return coords

def linePixels(pin0, pin1):
    length = int(np.hypot(pin1[0] - pin0[0], pin1[1] - pin0[1]))

    x = np.linspace(pin0[1], pin1[1], length)
    y = np.linspace(pin0[0], pin1[0], length)

    return (x.astype(np.int)-1, y.astype(np.int)-1)

In [ ]:
imgPath = "../"

In [ ]:
# Load image
image = cv2.imread(imgPath)

# Crop image
height, width = image.shape[0:2]
minEdge= min(height, width)
topEdge = int((height - minEdge)/2)
leftEdge = int((width - minEdge)/2)
imgCropped = image[topEdge:topEdge+minEdge, leftEdge:leftEdge+minEdge]

# Convert to grayscale
imgGray = cv2.cvtColor(imgCropped, cv2.COLOR_BGR2GRAY)

# Resize image
imgSized = cv2.resize(imgGray, (2*imgRadius + 1, 2*imgRadius + 1)) 

# Invert image
imgInverted = invertImage(imgSized)

# Mask image
imgMasked = maskImage(imgInverted, imgRadius)

coords = pinCoords(imgRadius, nPins)
height, width = imgMasked.shape[0:1]

i = 0
lines = []
previousPins = []
oldPin = initPin
lineMask = np.zeros((height, width))

for line in range(nLines):
    i += 1
    bestLine = 0
    oldCoord = coords[oldPin]

     for index in range(1, nPins):
        pin = (oldPin + index) % nPins

        coord = coords[pin]
        xLine, yLine = linePixels(oldCoord, coord)

           lineSum = np.sum(imgMasked[yLine, xLine])

        if (lineSum > bestLine) and not(pin in previousPins):
            bestLine = lineSum
            bestPin = pin

    if len(previousPins) >= minLoop:
        previousPins.pop(1)
    previousPins.append(bestPin)

    lineMask = lineMask * 1
    cv2.line(lineMask, oldCoord, coords[bestPin], lineWeight, lineWidth)
    imgMasked = np.subtract(imgMasked, lineMask)

    lines.append((oldPin, bestPin))

     if bestPin == oldPin:
        break

    oldPin = bestPin